In [1]:
import json
from itertools import islice
from ast import literal_eval

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

from skmultilearn.model_selection import iterative_train_test_split
from skmultilearn.model_selection.measures import get_combination_wise_output_matrix

In [2]:
tqdm.pandas()

In [3]:
df = pd.read_csv('generated_dataset/full_dataset_with_translations.csv')
##
print(df.columns)
##
for col in [
        'sectors', 'subpillars_2d', 'subpillars_1d', 'geo_location',
        'specific_needs_groups', 'age', 'gender', 'reliability',
        'affected_groups_level_0', 'affected_groups_level_1',
        'affected_groups_level_2', 'affected_groups_level_3', 'severity'
]:
    df[col] = df[col].apply(lambda x: [] if x != x else literal_eval(x))


##
def count_vals(df, col):
    vals = Counter()
    for e in df[col]:
        vals.update(e)
    return vals.most_common()

/home/abdullah/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3172: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Index(['entry_id', 'excerpt', 'analysis_framework_id', 'lead_id', 'project_id',
       'verified', 'sectors', 'subpillars_2d', 'subpillars_1d', 'geo_location',
       'specific_needs_groups', 'severity', 'info_date', 'reliability',
       'affected_groups_level_0', 'affected_groups_level_1',
       'affected_groups_level_2', 'affected_groups_level_3', 'age', 'gender',
       'source_type', 'url', 'website', 'lang', 'translation_en',
       'translation_fr', 'translation_es'],
      dtype='object')


In [4]:
df["analysis_framework_id"] = df["analysis_framework_id"].apply(lambda x: str(int(x)))

In [5]:
tagname_to_tagvalues = {
    'sectors': set(),
    'subpillars_2d': set(),
    'subpillars_1d': set(),
    'specific_needs_groups': set(),
    'severity': set(),
    'age': set(),
    'gender': set(),
    'affected_groups_level_0': set(),
    'affected_groups_level_1': set(),
    'affected_groups_level_2': set(),
    'affected_groups_level_3': set(),
    'reliability': set(),
    'analysis_framework_id': set(),
    'lang': set(),
}

In [6]:
def unique_tags(col):
    unique = set()
    for label in col:
        unique.update(label)
    return unique

In [7]:
for tagname in tagname_to_tagvalues:
    if tagname in ['analysis_framework_id', 'lang']:
        tagname_to_tagvalues[tagname] = set(df[tagname].unique().tolist())
    else:
        tagname_to_tagvalues[tagname] = unique_tags(df[tagname])

In [8]:
tagname_to_tagvalues

{'sectors': {'Agriculture',
  'Cross',
  'Education',
  'Food Security',
  'Health',
  'Livelihoods',
  'Logistics',
  'NOT_MAPPED',
  'Nutrition',
  'Protection',
  'Shelter',
  'WASH'},
 'subpillars_2d': {'At Risk->Number Of People At Risk',
  'At Risk->Risk And Vulnerabilities',
  'Capacities & Response->International Response',
  'Capacities & Response->Local Response',
  'Capacities & Response->National Response',
  'Capacities & Response->Number Of People Reached/Response Gaps',
  'Humanitarian Conditions->Coping Mechanisms',
  'Humanitarian Conditions->Living Standards',
  'Humanitarian Conditions->Number Of People In Need',
  'Humanitarian Conditions->Physical And Mental Well Being',
  'Impact->Driver/Aggravating Factors',
  'Impact->Impact On People',
  'Impact->Impact On Systems, Services And Networks',
  'Impact->Number Of People Affected',
  'NOT_MAPPED',
  'Priority Interventions->Expressed By Humanitarian Staff',
  'Priority Interventions->Expressed By Population',
  'Pri

In [9]:
classes = set()
for tagvalues in tagname_to_tagvalues.values():
    classes |= tagvalues 
len(classes) # total number of tags

126

In [10]:
classes = classes - {np.nan, None, "None"}
classes

{'1004',
 '1306',
 '136',
 '137',
 '1465',
 '273',
 '495',
 '537',
 '552',
 '699',
 '726',
 '799',
 '829',
 'Adult (18 to 59 years old)',
 'Affected',
 'Agriculture',
 'All',
 'Asylum Seekers',
 'At Risk->Number Of People At Risk',
 'At Risk->Risk And Vulnerabilities',
 'Capacities & Response->International Response',
 'Capacities & Response->Local Response',
 'Capacities & Response->National Response',
 'Capacities & Response->Number Of People Reached/Response Gaps',
 'Casualties->Dead',
 'Casualties->Injured',
 'Casualties->Missing',
 'Child Head of Household',
 'Children/Youth (5 to 17 years old)',
 'Chronically Ill',
 'Completely Reliable',
 'Context->Demography',
 'Context->Economy',
 'Context->Environment',
 'Context->Legal & Policy',
 'Context->Politics',
 'Context->Security & Stability',
 'Context->Socio Cultural',
 'Covid-19->Cases',
 'Covid-19->Contact Tracing',
 'Covid-19->Deaths',
 'Covid-19->Hospitalization & Care',
 'Covid-19->Restriction Measures',
 'Covid-19->Testing',


In [11]:
class_to_id = {cls:i for i, cls in enumerate(classes)}

In [12]:
class_to_id

{'Unaccompanied or Separated Children': 0,
 'Humanitarian Access->Population To Relief': 1,
 'Pendular': 2,
 'Covid-19->Testing': 3,
 'Migrants': 4,
 'Health': 5,
 '1306': 6,
 'Displacement->Intentions': 7,
 'Not Usually Reliable': 8,
 'Non Displaced': 9,
 'Female Head of Household': 10,
 'All': 11,
 'Protection': 12,
 'Returnees': 13,
 'Refugees': 14,
 'Completely Reliable': 15,
 'Capacities & Response->Local Response': 16,
 'Humanitarian Access->Physical Constraints': 17,
 'Priority Interventions->Expressed By Population': 18,
 'Impact->Impact On People': 19,
 'Host': 20,
 'Casualties->Missing': 21,
 'Context->Security & Stability': 22,
 'Priority Needs->Expressed By Humanitarian Staff': 23,
 'Minor Problem': 24,
 '799': 25,
 'Asylum Seekers': 26,
 'Information And Communication->Communication Means And Preferences': 27,
 'Humanitarian Conditions->Number Of People In Need': 28,
 'Context->Socio Cultural': 29,
 '495': 30,
 'At Risk->Risk And Vulnerabilities': 31,
 '699': 32,
 'Impact-

In [13]:
for tagvalues in tagname_to_tagvalues.values():
    for tagvalue in tagvalues:
        df[tagvalue] = 0

In [16]:
labels_2d = np.zeros((df.shape[0], len(classes)))
for row in tqdm(df.iterrows(), total=df.shape[0]):
    for tagname in tagname_to_tagvalues.keys():
        idx = row[0]
        yi = row[1][tagname]
        if yi in [np.nan, None, "None"]: continue
        if isinstance(yi, str):
            df.at[idx, yi] = 1
            labels_2d[idx, class_to_id[yi]] = 1
        elif isinstance(yi, list):
            for lbl in yi:
                if lbl in [np.nan, None, "None"]: continue
                df.at[idx, lbl] = 1
                labels_2d[idx, class_to_id[lbl]] = 1

  0%|          | 0/157948 [00:00<?, ?it/s]

In [17]:
labels_2d.sum(axis=0, dtype=int)

array([   814,    228,    275,   1622,  16516,  34291,  73756,    506,
          121,   7154,    722,  84606,  28021,   4577,  15348,   1205,
          259,   1743,    290,  12821,   6202,    272,   7926,   1686,
          199,     45,   2080,    222,   1339,   1751,   2186,  11245,
         5087,  15133,    126,    512,    855,  10001,   1207,   6870,
         5507,   1001,  27953,  20794,   5334,    948,   2095,   2776,
           88,   2596,    155,  16575,  12539,  97596,    116,    277,
         3419,    431,    347,   3819,   1287,   8543,  29699,   5072,
         2800,   5222,  23216,   4587,    192,    627,   3961,  16306,
        38148,    726,   2359,  79425,   3378,   1914,   2051,   2868,
         3472,  15654,   5010,   1852,   3114,   2143,   2364,   6946,
         2804,  12739,   3095,   3346,    663,    517,    770,  17244,
         1336,  49905,   3194,    671,  15420,     32,   8721,   1789,
         9839,  11470,  10142,   1659,   3497, 152858,  26671,  16423,
      

In [18]:
df[classes].sum().astype(int).sort_values().to_dict()

{'726': 32,
 '799': 45,
 'Unreliable': 46,
 'Stateless': 88,
 'Elderly Head of Household': 116,
 'Not Usually Reliable': 121,
 'Non Host': 126,
 'Information And Communication->Information Challenges And Barriers': 155,
 'Child Head of Household': 192,
 'Minor Problem': 199,
 'Single Women (including Widows)': 202,
 'Information And Communication->Communication Means And Preferences': 222,
 'Humanitarian Access->Population To Relief': 228,
 'Capacities & Response->Local Response': 259,
 'Casualties->Missing': 272,
 'Pendular': 275,
 'At Risk->Number Of People At Risk': 277,
 'Priority Interventions->Expressed By Population': 290,
 'Information And Communication->Knowledge And Info Gaps (Pop)': 304,
 'Displacement->Pull Factors': 347,
 'Not Affected': 431,
 'Displacement->Intentions': 506,
 'Covid-19->Hospitalization & Care': 512,
 'LGBTQI+': 517,
 'Covid-19->Contact Tracing': 627,
 'Casualties->Injured': 663,
 'Information And Communication->Knowledge And Info Gaps (Hum)': 671,
 'Femal

In [19]:
X_train, y_train, X_test, y_test = iterative_train_test_split(
    df.index.to_numpy().reshape(-1, 1), labels_2d, test_size=0.1)

In [20]:
X_train, y_train, X_val, y_val = iterative_train_test_split(X_train,
                                                            y_train,
                                                            test_size=0.1)

In [26]:
df_train = df[df.index.isin(X_train.reshape(-1))]
df_val = df[df.index.isin(X_val.reshape(-1))]
df_test = df[df.index.isin(X_test.reshape(-1))]

In [27]:
(df_train.index & df_val.index, df_train.index & df_test.index,
 df_test.index & df_val.index)

/tmp/ipykernel_264786/393006598.py:1: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  (df_train.index & df_val.index, df_train.index & df_test.index,
/tmp/ipykernel_264786/393006598.py:2: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  df_test.index & df_val.index)


(Int64Index([], dtype='int64'),
 Int64Index([], dtype='int64'),
 Int64Index([], dtype='int64'))

In [30]:
cols = [
    'entry_id', 'excerpt', 'analysis_framework_id', 'lead_id', 'project_id',
    'verified', 'sectors', 'subpillars_2d', 'subpillars_1d', 'geo_location',
    'specific_needs_groups', 'severity', 'info_date', 'age', 'gender',
    'reliability', 'affected_groups_level_0', 'affected_groups_level_1',
    'affected_groups_level_2', 'affected_groups_level_3', 'source_type', 'url',
    'website', 'lang', 'translation_en', 'translation_fr', 'translation_es'
]
df_train = df_train[cols]
df_val = df_val[cols]
df_test = df_test[cols]

In [31]:
df_train.to_csv("generated_dataset/train_v0.7.1.csv", index=None)
df_val.to_csv("generated_dataset/val_v0.7.1.csv", index=None)
df_test.to_csv("generated_dataset/test_v0.7.1.csv", index=None)

In [32]:
for tagname in tagname_to_tagvalues.keys():
    perc = df_train[tagname].apply(lambda x: x!= []).sum()
    perc = perc/df_train.shape[0]
    print(f"{tagname}: {1-perc:.2f}")

sectors: 0.22
subpillars_2d: 0.27
subpillars_1d: 0.53
specific_needs_groups: 0.95
severity: 0.70
age: 0.85
gender: 0.93
affected_groups_level_0: 0.46
affected_groups_level_1: 0.49
affected_groups_level_2: 0.67
affected_groups_level_3: 0.67
reliability: 0.01
analysis_framework_id: 0.00
lang: 0.00


In [33]:
for tagname in tagname_to_tagvalues.keys():
    perc = df_val[tagname].apply(lambda x: x!= []).sum()
    perc = perc/df_val.shape[0]
    print(f"{tagname}: {1-perc:.2f}")

sectors: 0.25
subpillars_2d: 0.32
subpillars_1d: 0.57
specific_needs_groups: 0.94
severity: 0.61
age: 0.86
gender: 0.94
affected_groups_level_0: 0.47
affected_groups_level_1: 0.50
affected_groups_level_2: 0.67
affected_groups_level_3: 0.67
reliability: 0.01
analysis_framework_id: 0.00
lang: 0.00


In [34]:
for tagname in tagname_to_tagvalues.keys():
    perc = df_test[tagname].apply(lambda x: x!= []).sum()
    perc = perc/df_test.shape[0]
    print(f"{tagname}: {1-perc:.2f}")

sectors: 0.21
subpillars_2d: 0.26
subpillars_1d: 0.58
specific_needs_groups: 0.95
severity: 0.69
age: 0.85
gender: 0.94
affected_groups_level_0: 0.49
affected_groups_level_1: 0.54
affected_groups_level_2: 0.70
affected_groups_level_3: 0.70
reliability: 0.02
analysis_framework_id: 0.00
lang: 0.00


In [35]:
for tagname in tagname_to_tagvalues.keys():
    perc = df[tagname].apply(lambda x: x!= []).sum()
    perc = perc/df.shape[0]
    print(f"{tagname}: {1-perc:.2f}")

sectors: 0.22
subpillars_2d: 0.27
subpillars_1d: 0.54
specific_needs_groups: 0.95
severity: 0.69
age: 0.85
gender: 0.93
affected_groups_level_0: 0.46
affected_groups_level_1: 0.50
affected_groups_level_2: 0.67
affected_groups_level_3: 0.68
reliability: 0.01
analysis_framework_id: 0.00
lang: 0.00


In [37]:
tagnames

['sectors',
 'subpillars_2d',
 'subpillars_1d',
 'specific_needs_groups',
 'severity',
 'age',
 'gender',
 'affected_groups_level_0',
 'affected_groups_level_1',
 'affected_groups_level_2',
 'affected_groups_level_3',
 'reliability',
 'analysis_framework_id',
 'lang']

In [39]:
tagnames = list(tagname_to_tagvalues.keys())
def merge_all_labels(x):
    res = []
    for t in x[tagnames[:-2]]:
        if isinstance(t, list):
            res.extend(t)
        elif isinstance(t, str):
            res.append(t)
    return res

df['target'] = df.progress_apply(merge_all_labels, axis=1)

  0%|          | 0/157948 [00:00<?, ?it/s]

In [40]:
df['target'].apply(lambda x: x==[]).sum()

94

In [41]:
df_bis = df[['entry_id', 'target', 'lead_id']].copy()

df_bis['count'] = df_bis['target'].apply(lambda x: len(x))

max_counts = df_bis[['lead_id', 'count']].groupby('lead_id',
                                                  as_index=False).max()
tagged_leads = max_counts[max_counts['count'] > 0]['lead_id'].tolist()

all_negative_ids = df_bis[
    df_bis.lead_id.isin(tagged_leads)
    & df_bis.target.apply(lambda x: len(x) == 0)].entry_id.unique().tolist()

In [42]:
df_tagged = df[~df.lead_id.isin(all_negative_ids)]
df_tagged.shape, df.shape

((157869, 154), (157948, 154))

In [43]:
df_tagged = df[~df.lead_id.isin(all_negative_ids)]
for tagname in tagname_to_tagvalues.keys():
    perc = df_tagged[tagname].apply(lambda x: x!= []).sum()
    perc = perc/df_tagged.shape[0]
    print(f"{tagname}: {1-perc:.2f}")

sectors: 0.22
subpillars_2d: 0.27
subpillars_1d: 0.54
specific_needs_groups: 0.95
severity: 0.69
age: 0.85
gender: 0.93
affected_groups_level_0: 0.46
affected_groups_level_1: 0.50
affected_groups_level_2: 0.67
affected_groups_level_3: 0.68
reliability: 0.01
analysis_framework_id: 0.00
lang: 0.00


In [44]:
df_train_tagged = df_train[df_train.index.isin(tagged_leads)]
for tagname in tagname_to_tagvalues.keys():
    perc = df_train_tagged[tagname].apply(lambda x: x!= []).sum()
    perc = perc/df_train_tagged.shape[0]
    print(f"{tagname}: {1-perc:.2f}")

sectors: 0.28
subpillars_2d: 0.33
subpillars_1d: 0.54
specific_needs_groups: 0.95
severity: 0.49
age: 0.90
gender: 0.96
affected_groups_level_0: 0.45
affected_groups_level_1: 0.48
affected_groups_level_2: 0.62
affected_groups_level_3: 0.63
reliability: 0.03
analysis_framework_id: 0.00
lang: 0.00


In [45]:
df_val_tagged = df_val[df_val.index.isin(tagged_leads)]
for tagname in tagname_to_tagvalues.keys():
    perc = df_val_tagged[tagname].apply(lambda x: x!= []).sum()
    perc = perc/df_val_tagged.shape[0]
    print(f"{tagname}: {1-perc:.2f}")

sectors: 0.26
subpillars_2d: 0.33
subpillars_1d: 0.57
specific_needs_groups: 0.96
severity: 0.49
age: 0.88
gender: 0.96
affected_groups_level_0: 0.45
affected_groups_level_1: 0.47
affected_groups_level_2: 0.61
affected_groups_level_3: 0.61
reliability: 0.02
analysis_framework_id: 0.00
lang: 0.00


In [46]:
df_test_tagged = df_test[df_test.index.isin(tagged_leads)]
for tagname in tagname_to_tagvalues.keys():
    perc = df_test_tagged[tagname].apply(lambda x: x!= []).sum()
    perc = perc/df_test_tagged.shape[0]
    print(f"{tagname}: {1-perc:.2f}")

sectors: 0.25
subpillars_2d: 0.30
subpillars_1d: 0.62
specific_needs_groups: 0.95
severity: 0.63
age: 0.87
gender: 0.96
affected_groups_level_0: 0.52
affected_groups_level_1: 0.54
affected_groups_level_2: 0.68
affected_groups_level_3: 0.68
reliability: 0.05
analysis_framework_id: 0.00
lang: 0.00
